# MIDI music generation with Transformer

In this notebook we'll be presenting various approaches for training a Transformer model that generates music in the form of MIDI (Musical Instrument Digital Interface) files.

Notebook presented for the A.A. 2023/2024 Deep Learning project.

Group members:

- Valerio Di Zio - valerio.dizio@studio.unibo.it
- Francesco Magnani - francesco.magnani14@studio.unibo.it
- Luca Rubboli - mail@diluca.it

## Installation of libraries

In [1]:
%%capture
!pip install gdown
!pip install pretty_midi
!pip install miditok
!pip install pretty_midi midi-clip
!pip install transformers
!pip install accelerate

## Dataset Download

In [2]:
%%capture
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
!unzip "maestro-v3.0.0-midi.zip"
!rm "maestro-v3.0.0-midi.zip"

dataset_path = "maestro-v3.0.0"

## Setup
Import of libraries and setup of downloaded files

In [4]:
import json
import numpy as np
import os
import random
import shutil
from pathlib import Path
from random import shuffle

import pretty_midi
import tensorflow as tf
from miditok import REMI, TokenizerConfig
from miditok.data_augmentation import augment_dataset
from miditok.utils import split_files_for_training
from tqdm import tqdm

In [7]:
# Paths to the files of the dataset

midi_paths = list(Path(dataset_path).resolve().glob("**/*.mid")) + list(Path(dataset_path).resolve().glob("**/*.midi"))

midis_dir = "midis"
os.makedirs(midis_dir, exist_ok=True)

for i, midi_path in enumerate(midi_paths):
  new_midi_path = os.path.join(midis_dir, f"{i}.midi")
  shutil.move(str(midi_path), new_midi_path)


midis = list(Path("midis").resolve().glob("**/*.mid")) + list(Path("midis").resolve().glob("**/*.midi"))

def sample():
  return str(random.choice(midis))

## Tokenizer
A tokenizer in the context of MIDI tokenization converts MIDI files into a sequence of **tokens** (e.g., integers or strings) that represent musical elements like notes, durations, velocities, and time shifts.

The Miditok library provides predefined tokenization methods (e.g. **REMI**) to translate MIDI events into tokens and vice versa, ensuring a structured and efficient representation of musical data.

### First approach to tokenization
Tokenization is probably one of the most important steps in the process.

For tokenization, in this specific example, a representation in the following form was used.

note-duration

Example: "C4-1.0, C4-1.0, G4-1.0, G4-1.0, A4-1.0, A4-1.0, G4-2.0, F4-1.0, F4-1.0, E4-1.0, E4-1.0, D4-1.0, D4-1.0, C4-2.0, G4-1.0, G4-1.0, F4-1.0, F4-1.0, E4-1.0, E4-1.0, D4-2.0, G4-1.0, G4-1.0, F4-1.0, F4-1.0, E4-1.0, E4-1.0, D4-2.0"

This allows each token to be assigned, 1 note.

A midi file, in fact, is much more complicated than this and by going about tokenizing differently, there is a risk of generating tokens that in sequence do not make sense.

By going to restrict the model so that we get for each token 1 note we make subsequent training and generation easier.

**This method unfortunately works worse than Miditok, consequently in this notebook Miditok will be used as the main tokenizer.**

In [33]:
STRING_TOKENIZATION = False
if STRING_TOKENIZATION:
  def find_midi_files(directory):
      """Recursively finds all MIDI files in the directory."""
      midi_files = []
      for root, _, files in os.walk(directory):
          for file in files:
              if file.endswith((".mid", ".midi")):
                  midi_files.append(os.path.join(root, file))
      return midi_files

  def midi_to_note_representation(file_path):
      """Converts a MIDI file into a note-duration representation."""
      try:
          midi_data = pretty_midi.PrettyMIDI(file_path)
          note_events = []

          for instrument in midi_data.instruments:
              for note in instrument.notes:
                  # Convert pitch to note name
                  note_name = pretty_midi.note_number_to_name(note.pitch)
                  duration = note.end - note.start
                  note_events.append(f"{note_name}-{duration:.1f}")

          return ", ".join(note_events)
      except Exception as e:
          print(f"Error in file conversion {file_path}: {e}")
          return None

  def create_dataset_from_midi(directory, output_file):
      """Creates a JSON dataset with the representation of notes from MIDI files."""
      dataset = {}
      midi_files = find_midi_files(directory)

      for midi_file in tqdm(midi_files):
          note_representation = midi_to_note_representation(midi_file)
          if note_representation:
              dataset[midi_file] = note_representation

      with open(output_file, "w") as json_file:
          json.dump(dataset, json_file, indent=4)

      print(f"Dataset created and saved in {output_file}")

  midi_directory = "./Maestro"
  output_dataset_file = "midi_dataset.json"


  create_dataset_from_midi(midi_directory, output_dataset_file)

  dataset_file = "midi_dataset.json"

  with open(dataset_file, "r") as json_file:
      dataset = json.load(json_file)
      maestro_dataset = list(dataset.values())

  from tensorflow.keras.preprocessing.text import Tokenizer

  tokenizer = Tokenizer(filters="", lower=False, split=",")
  tokenizer.fit_on_texts(maestro_dataset)
  tokenized_melodies = tokenizer.texts_to_sequences(maestro_dataset)

### Miditok Tokenization

In [9]:
# BEAT_RES defines the temporal resolution for tokenizing MIDI events,
# mapping beat intervals to the number of subdivisions per beat.
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}

TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 24,
    "special_tokens": ["PAD", "BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": True,
    "use_time_signatures": True,
    "use_programs": False,  # no multitrack
    "num_tempos": 32,
    "tempo_range": (50, 200),  # (min_tempo, max_tempo)
}

config = TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer = REMI(config)

### Load a trained tokenizer

In [11]:
!gdown 1SDRkoWwyuSl4udoCHdcitjLLm9d0kfxS
tokenizer = REMI(params=Path("maestro_tokenizer.json"))
print(f"Vocab size: {len(tokenizer)}")

Downloading...
From: https://drive.google.com/uc?id=1SDRkoWwyuSl4udoCHdcitjLLm9d0kfxS
To: /content/maestro_tokenizer.json
100% 1.66M/1.66M [00:00<00:00, 141MB/s]
Vocab size: 30000


### (Optional): train the tokenizer
This cell is optional because the tokenizer train is long, consequently as a matter of time it is skipped in favor of the previous cell that imports the already trained tokenizer

In [12]:
TRAIN_TOKENIZER = False
if TRAIN_TOKENIZER:
  tokenizer.train(vocab_size=30000, files_paths=midis)
  processed = [Path(f"{s}") for s in midis]
  print(len(processed))

## Split Dataset
Divide the Maestro dataset into Train, Valid and Test by also performing data augmentation

In [16]:
!gdown 1IQToXD9s8g4L-AlK-MY4qvGoLZ-p7bMw
!gdown 1DWjViUKpW07LfbGimlhhhGdK7oQaJpj-

all_ids_train = np.loadtxt("all_ids_maestro_train.txt").astype(np.int32)
all_ids_valid = np.loadtxt("all_ids_maestro_valid.txt").astype(np.int32)

Downloading...
From (original): https://drive.google.com/uc?id=1IQToXD9s8g4L-AlK-MY4qvGoLZ-p7bMw
From (redirected): https://drive.google.com/uc?id=1IQToXD9s8g4L-AlK-MY4qvGoLZ-p7bMw&confirm=t&uuid=2e9fdf42-4340-4666-9594-f962c18cf909
To: /content/all_ids_maestro_train.txt
100% 1.28G/1.28G [00:05<00:00, 226MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1DWjViUKpW07LfbGimlhhhGdK7oQaJpj-
From (redirected): https://drive.google.com/uc?id=1DWjViUKpW07LfbGimlhhhGdK7oQaJpj-&confirm=t&uuid=5853ad5a-750d-4f98-bcf6-d49540731c1a
To: /content/all_ids_maestro_valid.txt
100% 572M/572M [00:02<00:00, 223MB/s]


### (Optional): perform split

In [17]:
SPLIT_DATASET = False
if SPLIT_DATASET:

  total_num_files = len(processed)
  num_files_valid = round(total_num_files * 0.15)
  num_files_test = round(total_num_files * 0.15)
  shuffle(processed)
  midi_paths_valid = processed[:num_files_valid]
  midi_paths_test = processed[num_files_valid:num_files_valid + num_files_test]
  midi_paths_train = processed[num_files_valid + num_files_test:]

  # Chunk MIDIs and perform data augmentation on each subset independently

  for files_paths, subset_name in (
      (midi_paths_train, "train"),
      (midi_paths_valid, "valid"),
        (midi_paths_test, "test")
  ):
      print(files_paths[0])
      # Split the MIDIs into chunks of sizes approximately about 1024 tokens
      subset_chunks_dir = Path(f"Maestro_{subset_name}")

      split_files_for_training(
          files_paths=files_paths,
          tokenizer=tokenizer,
          save_dir=subset_chunks_dir,
          max_seq_len=1024,
          num_overlap_bars=2,
      )

      # Perform data augmentation
      augment_dataset(
          subset_chunks_dir,
          pitch_offsets=[-12, 12],
          velocity_offsets=[-4, 4],
          duration_offsets=[-0.5, 0.5],
      )
  midi_paths_train = list(Path("Maestro_train").glob("**/*.mid")) + list(Path("Maestro_train").glob("**/*.midi"))
  midi_paths_valid = list(Path("Maestro_valid").glob("**/*.mid")) + list(Path("Maestro_valid").glob("**/*.midi"))
  midi_paths_test = list(Path("Maestro_test").glob("**/*.mid")) + list(Path("Maestro_test").glob("**/*.midi"))


In [18]:
if SPLIT_DATASET:
  def midi_valid(midi) -> bool:
      if any(ts.numerator != 4 for ts in midi.time_signature_changes):
          return False  # time signature different from 4/*, 4 beats per bar
      return True

  if os.path.exists("tokenized"):
    shutil.rmtree("tokenized")

  for dir in ("train", "valid", "test"):
      tokenizer.tokenize_dataset(
          Path(f"Maestro_{dir}"),
          Path(f"tokenized_{dir}"),
          midi_valid,
      )

In [19]:
if SPLIT_DATASET:
  def read_json(path: str) -> dict:
    with open(path, "r") as f:
      return json.load(f)

  def read_json_files(json_file_paths):
      """Reads a list of JSON files and returns a list of objects.
      Args:
          json_file_paths: A list of file paths to JSON files.
      Returns:
          A list of objects, where each object represents the data from a JSON file.
          Returns an empty list if any error occurs during file processing.
      """
      objects = []

      for file_path in tqdm(json_file_paths):
          try:
              objects.append(read_json(file_path))
          except FileNotFoundError:
              print(f"Error: File not found - {file_path}")
              return [] # Return empty list on error
          except json.JSONDecodeError:
              print(f"Error decoding JSON in file: {file_path}")
              return [] # Return empty list on error
      return objects

In [20]:
if SPLIT_DATASET:
  tokenized_train = list(Path("tokenized_train").resolve().glob("**/*.json"))
  data_objects_train = read_json_files(tokenized_train)

  tokenized_valid = list(Path("tokenized_valid").resolve().glob("**/*.json"))
  data_objects_valid = read_json_files(tokenized_valid)

  tokenized_test = list(Path("tokenized_test").resolve().glob("**/*.json"))
  data_objects_test = read_json_files(tokenized_test)


  if data_objects_train:
      print(f"\nSuccessfully read {len(data_objects_train)} training JSON files.")
  else:
      print("Error reading JSON files.")

In [21]:
if SPLIT_DATASET:
  encoded_train = [np.array(song["ids"][0]) for song in data_objects_train]
  encoded_valid = [np.array(song["ids"][0]) for song in data_objects_valid]
  encoded_test = [np.array(song["ids"][0]) for song in data_objects_test]

  all_ids_train = np.concatenate(encoded_train)
  all_ids_valid = np.concatenate(encoded_valid)
  all_ids_test = np.concatenate(encoded_test)

### Create Tensorflow Dataset

In [23]:
ids_dataset_train = tf.data.Dataset.from_tensor_slices(all_ids_train)
ids_dataset_valid = tf.data.Dataset.from_tensor_slices(all_ids_valid)

### Convert into sequences

In [24]:
SEQ_LENGTH = 1024

sequences_train = ids_dataset_train.batch(SEQ_LENGTH+1, drop_remainder=True)
sequences_valid = ids_dataset_valid.batch(SEQ_LENGTH+1, drop_remainder=True)

### Preparing labels

In [25]:
def split_input_target(sequence):
    # Convert to float32
    input_seq = tf.cast(sequence[:-1], tf.int32)
    target_seq = tf.cast(sequence[1:], tf.int32)
    return input_seq, target_seq


train_ds = sequences_train.map(split_input_target)
valid_ds = sequences_valid.map(split_input_target)

### Create training batches
Buffer size to shuffle the dataset
(TF data is designed to work with possibly infinite sequences, so it doesn't attempt to shuffle the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements).

In [26]:
BATCH_SIZE = 16
BUFFER_SIZE = 10000

train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
valid_ds = (
    valid_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

## Building Model

In [27]:
from transformers import TFAutoModelForCausalLM, MistralConfig

# Define the model configuration
model_config = MistralConfig(
    vocab_size=len(tokenizer),
    hidden_size=512,
    intermediate_size=1024,
    num_hidden_layers=8,
    num_attention_heads=8,
    num_key_value_heads=4,
    sliding_window=256,
    max_position_embeddings=8192,
    pad_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)

# Initialize the TensorFlow model
model = TFAutoModelForCausalLM.from_config(model_config)
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss,
              optimizer="adam",
              weighted_metrics=["sparse_categorical_accuracy"],
              jit_compile=True,
              )

## Train

In [29]:
!gdown 1qoTIkmhmj0z4854FFofz02KLk5DnyqB1
!unzip NESGEN_Maestro15.zip

model.from_pretrained("nesgen-maestro0612")

Downloading...
From (original): https://drive.google.com/uc?id=1qoTIkmhmj0z4854FFofz02KLk5DnyqB1
From (redirected): https://drive.google.com/uc?id=1qoTIkmhmj0z4854FFofz02KLk5DnyqB1&confirm=t&uuid=0a6995a8-f47a-4efe-9393-ac11e4853015
To: /content/NESGEN_Maestro15.zip
100% 184M/184M [00:00<00:00, 187MB/s]
Archive:  NESGEN_Maestro15.zip
replace nesgen-maestro0612/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

All model checkpoint layers were used when initializing TFMistralForCausalLM.

All the layers of TFMistralForCausalLM were initialized from the model checkpoint at nesgen-maestro0612.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMistralForCausalLM for predictions without further training.


### (Optional): Train model
Training of this model is made optional, free colab does not allow for this training.

In [30]:
TRAIN = False
if TRAIN:
  EPOCHS = 15

  history = model.fit(
      train_ds,
      epochs=EPOCHS,
      validation_data=valid_ds,
  )

## Generation

### Select a seed from the test set

In [31]:
def get_seed():
    for seed_ids, _ in valid_ds.take(1):
      seed = seed_ids
    return seed[0]

### Generate and dump MIDI


In [32]:
dump_seed = False

seed = get_seed()
input_ids = tf.convert_to_tensor(seed)  # Assuming seed is a 1D tensor of token IDs
input_ids = tf.expand_dims(input_ids, 0)  # Add an extra dimension to represent batch size

if dump_seed:
    midi = tokenizer.decode([seed])
    midi.dump_midi("seed.mid")

# Generate continuation
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=256,  # Maximum length of generated sequence
    num_return_sequences=1,  # Number of sequences to return
    do_sample=True,  # Use sampling (True) or greedy decoding (False)
    temperature=0.7  # Sampling temperature (lower is more conservative)
)

input_length = input_ids.shape[1]
generated_tokens = outputs[:, input_length:] # skip seed

# Decode and print the generated text
generated = tokenizer.decode([generated_tokens[0]])
generated.dump_midi("nesgen-generation.mid")

# Fine tuning with NesDB
The purpose of the project is to generate music similar to that of the old NES console games.

Consequently, it is necessary to perform fine tuning of the model trained on master using the NesDB dataset

## Download Dataset

In [53]:
%%capture
!gdown 1gIli7G1wu0QWDLzRc-CPWB8C4Hu0XVn3
!unzip nesmdb_midi.zip
!rm nesmdb_midi.zip

## Tokenizer

In [35]:
%%capture
!gdown 1ul-khIle1ue9oUGqb1Z7qHxk7LSuxrWw
tokenizer = REMI(params=Path("nes_tokenizer.json"))
print(f"Vocab size: {len(tokenizer)}")

## Split Dataset

In [36]:
# Download the files
!gdown 1iVwuKKT2sIezyVIDB9STL--FYZf0XcxF # train
!gdown 1FoOC3x4GbdDe0fwlzE1Zs8fqZXyFQCIA # valid

Downloading...
From (original): https://drive.google.com/uc?id=1iVwuKKT2sIezyVIDB9STL--FYZf0XcxF
From (redirected): https://drive.google.com/uc?id=1iVwuKKT2sIezyVIDB9STL--FYZf0XcxF&confirm=t&uuid=1b7227b9-72ac-4f1f-a68a-017b98a47dc4
To: /content/all_ids_nes_train.txt
100% 128M/128M [00:01<00:00, 90.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FoOC3x4GbdDe0fwlzE1Zs8fqZXyFQCIA
To: /content/all_ids_nes_valid.txt
100% 60.1M/60.1M [00:01<00:00, 41.9MB/s]


In [39]:
all_ids_train = np.loadtxt("all_ids_nes_train.txt").astype(np.int32)
all_ids_valid = np.loadtxt("all_ids_nes_valid.txt").astype(np.int32)

### (Optional): perform split

In [40]:
if SPLIT_DATASET:
  nesmdb_paths = list(Path("nesmdb_midi").resolve().glob("**/*.mid")) + list(Path("nesmdb_midi").resolve().glob("**/*.midi"))

  nesmdb_dir = "nesmdb"
  os.makedirs(nesmdb_dir, exist_ok=True)

  for i, midi_path in enumerate(nesmdb_paths):
    new_midi_path = os.path.join(nesmdb_dir, f"{i}.midi")
    shutil.move(str(midi_path), new_midi_path)

  nes_midis = list(Path("nesmdb").resolve().glob("**/*.mid")) + list(Path("nesmdb").resolve().glob("**/*.midi"))

In [41]:
if SPLIT_DATASET:
  total_num_files = len(processed)

  num_files_valid = round(total_num_files * 0.15)
  shuffle(processed)
  midi_paths_valid = processed[:num_files_valid]
  midi_paths_train = processed[num_files_valid + num_files_test:]

  # Chunk MIDIs and perform data augmentation on each subset independently
  for files_paths, subset_name in (
      (midi_paths_train, "train"), (midi_paths_valid, "valid")
  ):
      print(files_paths[0])
      # Split the MIDIs into chunks of sizes approximately about 1024 tokens
      subset_chunks_dir = Path(f"Nesmdb_{subset_name}")

      split_files_for_training(
          files_paths=files_paths,
          tokenizer=tokenizer,
          save_dir=subset_chunks_dir,
          max_seq_len=1024,
          num_overlap_bars=2,
      )

      # Perform data augmentation
      augment_dataset(
          subset_chunks_dir,
          pitch_offsets=[-12, 12],
          velocity_offsets=[-4, 4],
          duration_offsets=[-0.5, 0.5],
      )

  midi_paths_train = list(Path("Nesmdb_train").glob("**/*.mid")) + list(Path("Nesmdb_train").glob("**/*.midi"))
  midi_paths_valid = list(Path("Nesmdb_valid").glob("**/*.mid")) + list(Path("Nesmdb_valid").glob("**/*.midi"))


In [42]:
if SPLIT_DATASET:
  def midi_valid(midi) -> bool:
      if any(ts.numerator != 4 for ts in midi.time_signature_changes):
          return False  # time signature different from 4/*, 4 beats per bar
      return True

  if os.path.exists("tokenized_train"):
    shutil.rmtree("tokenized_train")
  if os.path.exists("tokenized_valid"):
    shutil.rmtree("tokenized_valid")

  for dir in ("train", "valid"):
      tokenizer.tokenize_dataset(
          Path(f"Nesmdb_{dir}"),
          Path(f"tokenized_{dir}"),
          midi_valid,
      )

In [43]:
if SPLIT_DATASET:
  tokenized_train = list(Path("tokenized_train").resolve().glob("**/*.json"))
  data_objects_train = read_json_files(tokenized_train)

  tokenized_valid = list(Path("tokenized_valid").resolve().glob("**/*.json"))
  data_objects_valid = read_json_files(tokenized_valid)


  if data_objects_train:
      print(f"\nSuccessfully read {len(data_objects_train)} training JSON files.")
  else:
      print("Error reading JSON files.")

In [44]:
if SPLIT_DATASET:
  encoded_train = [np.array(song["ids"][0]) for song in data_objects_train]
  encoded_valid = [np.array(song["ids"][0]) for song in data_objects_valid]

  all_ids_train = np.concatenate(encoded_train)
  all_ids_valid = np.concatenate(encoded_valid)

### Convert Tensorflow Dataset

In [45]:
ids_dataset_train = tf.data.Dataset.from_tensor_slices(all_ids_train)
ids_dataset_valid = tf.data.Dataset.from_tensor_slices(all_ids_valid)

### Convert into sequences

In [46]:
SEQ_LENGTH = 1024

sequences_train = ids_dataset_train.batch(SEQ_LENGTH+1, drop_remainder=True)
sequences_valid = ids_dataset_valid.batch(SEQ_LENGTH+1, drop_remainder=True)

### Preparing labels

In [47]:
def split_input_target(sequence):
    # Convert to float32
    input_seq = tf.cast(sequence[:-1], tf.int32)
    target_seq = tf.cast(sequence[1:], tf.int32)
    return input_seq, target_seq


train_ds = sequences_train.map(split_input_target)
valid_ds = sequences_valid.map(split_input_target)

### Creating training batches

In [48]:
# Batch size

BATCH_SIZE = 16

BUFFER_SIZE = 10000



train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
valid_ds = (
    valid_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

## Get the model

In [50]:
!gdown 1FfofRGCzuD6x54Q3mF2c_f50wuzbPp_i

!unzip NESGEN_Nesmdb5.zip
!rm -rf NESGEN_Nesmdb5.zip

Downloading...
From (original): https://drive.google.com/uc?id=1FfofRGCzuD6x54Q3mF2c_f50wuzbPp_i
From (redirected): https://drive.google.com/uc?id=1FfofRGCzuD6x54Q3mF2c_f50wuzbPp_i&confirm=t&uuid=cf7ddba4-54a1-4aba-abda-491db6223310
To: /content/NESGEN_Nesmdb5.zip
100% 184M/184M [00:04<00:00, 39.6MB/s]
Archive:  NESGEN_Nesmdb5.zip
   creating: nesgen_v1/
  inflating: nesgen_v1/tf_model.h5   
  inflating: nesgen_v1/config.json   
  inflating: nesgen_v1/generation_config.json  


## Train

In [54]:
from transformers import TFAutoModelForCausalLM
model = TFAutoModelForCausalLM.from_pretrained("./nesgen-maestro0612")

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss,
              optimizer="adam",
              weighted_metrics=["sparse_categorical_accuracy"],
              jit_compile=True,
              )

All model checkpoint layers were used when initializing TFMistralForCausalLM.

All the layers of TFMistralForCausalLM were initialized from the model checkpoint at ./nesgen-maestro0612.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMistralForCausalLM for predictions without further training.


In [55]:
EPOCHS = 5
if TRAIN:
  history = model.fit(
      train_ds,
      epochs=EPOCHS,
      validation_data=valid_ds,
  )

In [56]:
model.from_pretrained("nesgen_v1")

All model checkpoint layers were used when initializing TFMistralForCausalLM.

All the layers of TFMistralForCausalLM were initialized from the model checkpoint at nesgen_v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMistralForCausalLM for predictions without further training.


In [58]:
dump_seed = False

seed = get_seed()
input_ids = tf.convert_to_tensor(seed)  # Assuming seed is a 1D tensor of token IDs
input_ids = tf.expand_dims(input_ids, 0)  # Add an extra dimension to represent batch size

if dump_seed:
    midi = tokenizer.decode([seed])
    midi.dump_midi("seed.mid")

# Generate continuation
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=256,  # Maximum length of generated sequence
    num_return_sequences=1,  # Number of sequences to return
    do_sample=True,  # Use sampling (True) or greedy decoding (False)
    temperature=0.7  # Sampling temperature (lower is more conservative)
)

input_length = input_ids.shape[1]
generated_tokens = outputs[:, input_length:] # skip seed

# Decode and print the generated text
generated = tokenizer.decode([generated_tokens[0]])
generated.dump_midi("nesgen-nesdb-generation.mid")